In [1]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn =  MongoClient(host='localhost', port=27017, connect=False)
db = conn.PaperData
Spectrogram_Collection  =  db.get_collection('AudioSpectrogram_100thTag')
#LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')

In [2]:
#Audio Para
_mel_scale = 96
_time_len = 1292
_channels = 1

In [3]:
#Tag para
_num_Tags = 50
_num_neural_TagFC1 = 512
_num_neural_TagFC2 = 256
_num_neural_TagFC3 = 2 #for soft max

In [4]:
_num_Tags = 50
_epchos = 200
_stopEpcho = 20
_batchSize = 32

In [5]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
    
x_train = read_json('x_train_'+str(_num_Tags)+'d_80p_sub.json')
x_test = read_json('x_test_'+str(_num_Tags)+'d_80p_sub.json')
y_train = read_json('y_train_'+str(_num_Tags)+'d_80p_sub.json')
y_test = read_json('y_test_'+str(_num_Tags)+'d_80p_sub.json')

In [6]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

Using TensorFlow backend.


In [7]:
Spectrogram_Input = Input(name= 'AudioInput',shape=(_mel_scale, _time_len, _channels)) #(None(1), 96, 1292, 1)
CL_1 = Conv2D(128, kernel_size=(3, 3),padding='same', name = "CL_1")(Spectrogram_Input) #(None, 96, 1292, 128) 
BN_1 = BatchNormalization()(CL_1)
AL_1 = ELU()(BN_1)
MP_1 = MaxPool2D(pool_size= (2,4), name = "MPL_1")(AL_1) #(None, 48, 323, 128)
DP_1 = Dropout(0.5)(MP_1)

CL_2 = Conv2D(384, kernel_size=(3, 3),padding='same', name = "CL_2")(DP_1) # (None, 48, 323, 384) 
BN_2 = BatchNormalization()(CL_2)
AL_2 = ELU()(BN_2)
MP_2 = MaxPool2D(pool_size= (4,5), name = "MPL_2")(AL_2) # (None, 12, 64, 384) 
DP_2 = Dropout(0.5)(MP_2)

CL_3 = Conv2D(768, kernel_size=(3, 3),padding='same', name = "CL_3")(DP_2) #(None, 12, 64, 768) 
BN_3 = BatchNormalization()(CL_3)
AL_3 = ELU()(BN_3)
MP_3 = MaxPool2D(pool_size= (3,8), name = "MPL_3")(AL_3) #(None, 4, 8, 768)
DP_3 = Dropout(0.5)(MP_3)

CL_4 = Conv2D(2048, kernel_size=(3, 3),padding='same', name = "CL_4")(DP_3) #(None, 4, 8, 2048) 
BN_4 = BatchNormalization()(CL_4)
AL_4 = ELU()(BN_4)
MP_4 = MaxPool2D(pool_size= (4,8), name = "MPL_4")(AL_4)#(None, 1, 1, 2048) 
DP_4 = Dropout(0.5)(MP_4)

AF = Flatten()(DP_4) # (None, 2048) 
#pseudo-model for sheck summary
#Softmax = Dense(_num_Tags, kernel_initializer='he_normal',activation='softmax')(AF)
#Audio_model = Model(inputs=[Spectrogram_Input], outputs=[Softmax])

In [8]:
Output_Layers = []
# add MLT classifer
for i in range(_num_Tags):
    MTL_FC1 = Dense(_num_neural_TagFC1, activation='tanh', name='Tag_FC1_'+str(i+1))(AF)
    MTL_FC2 = Dense(_num_neural_TagFC2, activation='tanh', name='Tag_FC2_'+str(i+1))(MTL_FC1)
    MTL_FC3 = Dense( 2, activation='softmax', name='Tag_'+str(i+1))(MTL_FC2)
    Output_Layers.append(MTL_FC3)

Audio_model = Model(inputs=Spectrogram_Input, outputs=Output_Layers)

In [9]:
Audio_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
AudioInput (InputLayer)         (None, 96, 1292, 1)  0                                            
__________________________________________________________________________________________________
CL_1 (Conv2D)                   (None, 96, 1292, 128 1280        AudioInput[0][0]                 
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 96, 1292, 128 512         CL_1[0][0]                       
__________________________________________________________________________________________________
elu_1 (ELU)                     (None, 96, 1292, 128 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
MPL_1 (Max

In [10]:
from keras.utils import Sequence
import json
import codecs
class AudioDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels,batch_size):
        self.text_filenames, self.labels = text_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.text_filenames)/ (self.batch_size)))

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename_List =  self.text_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #Text_batch_x = [] 
        Audio_batch_x = []
        for filename in filename_List:  
            Audio_x = np.array(Spectrogram_Collection.find_one({"Filename":filename})['Spectrogram']).reshape((_mel_scale,_time_len,_channels))
            Audio_batch_x.append(Audio_x)
            
        MLT_y = [[] for i in range(_num_Tags)]
        
        y_List = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        for y in y_List:
            for i in range(len(y)):
                gt = y[i]
                binary_gt = 0 if gt == 1 else 1
                MLT_y[i].append([gt, binary_gt])
        for i in range(len(MLT_y)):
            MLT_y[i] = np.array(MLT_y[i])
        
        return np.array(Audio_batch_x), MLT_y
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [11]:
ADG_train = AudioDataGenerator(x_train, y_train,_batchSize)
ADG_test = AudioDataGenerator(x_test, y_test,_batchSize)

In [12]:
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback(Callback):
    def __init__(self,y_test,SDG_test,log_name):
        self.y_test = y_test
        self.SDG_test = SDG_test
        self.log_name = log_name
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        Skip_Tag = []
        
        
        y_pred = self.model.predict_generator(self.SDG_test) # #tags(list)x(#data x 2(binary)) 
        
        #transform y_pred to #data x tags 
        y_pred_tag = []
        
        _numTest = len(self.y_test)
        _numTags = len(self.y_test[0])
        for data_index in range(_numTest):
            tag_result_thisdata = []
            for tag_index in range(_num_Tags):
                if tag_index in Skip_Tag:
                    continue
                if y_pred[tag_index][data_index][0] >= y_pred[tag_index][data_index][1]:
                    tag_result_thisdata.append(1)
                else:
                    tag_result_thisdata.append(0)
            y_pred_tag.append(tag_result_thisdata)
        AUC_this_epoch = roc_auc_score(y_test ,y_pred_tag)
        logs[self.log_name] = AUC_this_epoch
        print('%s : %s '% (self.log_name,(str(round(AUC_this_epoch,5)))))
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [13]:
AUC_test_callback =AUC_callback(y_test,ADG_test,'AUC_test')
AUC_train_callback =AUC_callback(y_train,ADG_train,'AUC_train')

In [14]:
import time
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_time_end = time.time()
        self.times.append(self.epoch_time_end - self.epoch_time_start)
        logs['Timer'] = self.epoch_time_end - self.epoch_time_start
time_callback = TimeHistory()

In [15]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=_stopEpcho, verbose=1)

In [16]:
x,y = ADG_train.getitem(0)

In [17]:
x.shape

(32, 96, 1292, 1)

In [18]:
len(y)

50

In [19]:
y[0]

array([[1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [1, 0],
       [0, 1],
       [0, 1]])

In [20]:
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

In [21]:
import keras_metrics
precision = keras_metrics.precision(label = 1)
recall = keras_metrics.recall(label = 0)

In [22]:
num_training_samples = len(x_train)
num_validation_samples = len(x_test)

In [23]:
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(Audio_model, gpus=2)
parallel_model.compile(loss="binary_crossentropy",
              optimizer='adam',
              metrics=['acc'])
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= _epchos,
                    verbose=1,
                    validation_data= ADG_test,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [early_stopping,time_callback],
                    #class_weight = class_weight
                    )

Epoch 1/200


/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo

640/641 [============================>.] - ETA: 0s - loss: 20.9994 - Tag_1_loss: 0.6651 - Tag_2_loss: 0.6473 - Tag_3_loss: 0.6879 - Tag_4_loss: 0.6809 - Tag_5_loss: 0.3996 - Tag_6_loss: 0.5017 - Tag_7_loss: 0.6122 - Tag_8_loss: 0.5583 - Tag_9_loss: 0.3542 - Tag_10_loss: 0.5343 - Tag_11_loss: 0.5412 - Tag_12_loss: 0.8735 - Tag_13_loss: 0.5202 - Tag_14_loss: 0.4775 - Tag_15_loss: 0.3444 - Tag_16_loss: 0.3623 - Tag_17_loss: 0.4742 - Tag_18_loss: 0.5038 - Tag_19_loss: 0.4376 - Tag_20_loss: 0.2881 - Tag_21_loss: 0.5404 - Tag_22_loss: 0.4560 - Tag_23_loss: 0.2771 - Tag_24_loss: 0.4319 - Tag_25_loss: 0.4113 - Tag_26_loss: 0.4194 - Tag_27_loss: 0.4165 - Tag_28_loss: 0.3882 - Tag_29_loss: 0.4818 - Tag_30_loss: 0.3573 - Tag_31_loss: 0.3558 - Tag_32_loss: 0.3172 - Tag_33_loss: 0.4407 - Tag_34_loss: 0.2790 - Tag_35_loss: 0.3583 - Tag_36_loss: 0.3556 - Tag_37_loss: 0.4055 - Tag_38_loss: 0.2102 - Tag_39_loss: 0.3921 - Tag_40_loss: 0.3167 - Tag_41_loss: 0.4087 - Tag_42_loss: 0.3281 - Tag_43_loss: 0.3

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


641/641 [==============================] - 340s 530ms/step - loss: 20.9965 - Tag_1_loss: 0.6650 - Tag_2_loss: 0.6472 - Tag_3_loss: 0.6878 - Tag_4_loss: 0.6809 - Tag_5_loss: 0.3995 - Tag_6_loss: 0.5017 - Tag_7_loss: 0.6122 - Tag_8_loss: 0.5584 - Tag_9_loss: 0.3540 - Tag_10_loss: 0.5344 - Tag_11_loss: 0.5413 - Tag_12_loss: 0.8722 - Tag_13_loss: 0.5205 - Tag_14_loss: 0.4774 - Tag_15_loss: 0.3443 - Tag_16_loss: 0.3623 - Tag_17_loss: 0.4742 - Tag_18_loss: 0.5038 - Tag_19_loss: 0.4374 - Tag_20_loss: 0.2881 - Tag_21_loss: 0.5404 - Tag_22_loss: 0.4566 - Tag_23_loss: 0.2771 - Tag_24_loss: 0.4317 - Tag_25_loss: 0.4112 - Tag_26_loss: 0.4192 - Tag_27_loss: 0.4170 - Tag_28_loss: 0.3880 - Tag_29_loss: 0.4819 - Tag_30_loss: 0.3567 - Tag_31_loss: 0.3556 - Tag_32_loss: 0.3170 - Tag_33_loss: 0.4409 - Tag_34_loss: 0.2790 - Tag_35_loss: 0.3582 - Tag_36_loss: 0.3552 - Tag_37_loss: 0.4056 - Tag_38_loss: 0.2101 - Tag_39_loss: 0.3921 - Tag_40_loss: 0.3168 - Tag_41_loss: 0.4086 - Tag_42_loss: 0.3280 - Tag_43_l

...

 56/641 [=>............................] - ETA: 4:34 - loss: 14.3477 - Tag_1_loss: 0.5100 - Tag_2_loss: 0.4712 - Tag_3_loss: 0.5065 - Tag_4_loss: 0.4159 - Tag_5_loss: 0.2019 - Tag_6_loss: 0.2503 - Tag_7_loss: 0.5608 - Tag_8_loss: 0.4584 - Tag_9_loss: 0.2002 - Tag_10_loss: 0.4237 - Tag_11_loss: 0.4032 - Tag_12_loss: 0.1632 - Tag_13_loss: 0.4127 - Tag_14_loss: 0.3422 - Tag_15_loss: 0.1769 - Tag_16_loss: 0.2550 - Tag_17_loss: 0.3570 - Tag_18_loss: 0.4375 - Tag_19_loss: 0.2846 - Tag_20_loss: 0.1511 - Tag_21_loss: 0.3336 - Tag_22_loss: 0.3312 - Tag_23_loss: 0.1341 - Tag_24_loss: 0.2892 - Tag_25_loss: 0.2444 - Tag_26_loss: 0.3308 - Tag_27_loss: 0.3408 - Tag_28_loss: 0.2880 - Tag_29_loss: 0.4420 - Tag_30_loss: 0.1106 - Tag_31_loss: 0.2482 - Tag_32_loss: 0.1618 - Tag_33_loss: 0.4040 - Tag_34_loss: 0.1719 - Tag_35_loss: 0.2202 - Tag_36_loss: 0.3077 - Tag_37_loss: 0.3710 - Tag_38_loss: 0.1417 - Tag_39_loss: 0.2512 - Tag_40_loss: 0.2324 - Tag_41_loss: 0.3624 - Tag_42_loss: 0.1807 - Tag_43_loss: 0

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


640/641 [============================>.] - ETA: 0s - loss: 14.1650 - Tag_1_loss: 0.5188 - Tag_2_loss: 0.4810 - Tag_3_loss: 0.5023 - Tag_4_loss: 0.3931 - Tag_5_loss: 0.2038 - Tag_6_loss: 0.2383 - Tag_7_loss: 0.5576 - Tag_8_loss: 0.4676 - Tag_9_loss: 0.1914 - Tag_10_loss: 0.4237 - Tag_11_loss: 0.3942 - Tag_12_loss: 0.1478 - Tag_13_loss: 0.4182 - Tag_14_loss: 0.3350 - Tag_15_loss: 0.1668 - Tag_16_loss: 0.2657 - Tag_17_loss: 0.3655 - Tag_18_loss: 0.4331 - Tag_19_loss: 0.2676 - Tag_20_loss: 0.1559 - Tag_21_loss: 0.3213 - Tag_22_loss: 0.3285 - Tag_23_loss: 0.1412 - Tag_24_loss: 0.2800 - Tag_25_loss: 0.2389 - Tag_26_loss: 0.3286 - Tag_27_loss: 0.3361 - Tag_28_loss: 0.2888 - Tag_29_loss: 0.4270 - Tag_30_loss: 0.0879 - Tag_31_loss: 0.2268 - Tag_32_loss: 0.1657 - Tag_33_loss: 0.3839 - Tag_34_loss: 0.1726 - Tag_35_loss: 0.2143 - Tag_36_loss: 0.3093 - Tag_37_loss: 0.3578 - Tag_38_loss: 0.1333 - Tag_39_loss: 0.2527 - Tag_40_loss: 0.2273 - Tag_41_loss: 0.3580 - Tag_42_loss: 0.1817 - Tag_43_loss: 0.2

/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo/topology.py:149: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "
/usr/local/lib/python3.6/dist-packages/pymongo

641/641 [==============================] - 315s 492ms/step - loss: 14.1656 - Tag_1_loss: 0.5194 - Tag_2_loss: 0.4808 - Tag_3_loss: 0.5022 - Tag_4_loss: 0.3935 - Tag_5_loss: 0.2037 - Tag_6_loss: 0.2383 - Tag_7_loss: 0.5577 - Tag_8_loss: 0.4675 - Tag_9_loss: 0.1913 - Tag_10_loss: 0.4236 - Tag_11_loss: 0.3942 - Tag_12_loss: 0.1477 - Tag_13_loss: 0.4181 - Tag_14_loss: 0.3349 - Tag_15_loss: 0.1669 - Tag_16_loss: 0.2657 - Tag_17_loss: 0.3655 - Tag_18_loss: 0.4330 - Tag_19_loss: 0.2677 - Tag_20_loss: 0.1558 - Tag_21_loss: 0.3212 - Tag_22_loss: 0.3285 - Tag_23_loss: 0.1415 - Tag_24_loss: 0.2798 - Tag_25_loss: 0.2386 - Tag_26_loss: 0.3286 - Tag_27_loss: 0.3359 - Tag_28_loss: 0.2886 - Tag_29_loss: 0.4271 - Tag_30_loss: 0.0878 - Tag_31_loss: 0.2270 - Tag_32_loss: 0.1659 - Tag_33_loss: 0.3843 - Tag_34_loss: 0.1726 - Tag_35_loss: 0.2144 - Tag_36_loss: 0.3091 - Tag_37_loss: 0.3579 - Tag_38_loss: 0.1333 - Tag_39_loss: 0.2527 - Tag_40_loss: 0.2276 - Tag_41_loss: 0.3580 - Tag_42_loss: 0.1819 - Tag_43_l

In [24]:
small_ADG = AudioDataGenerator(x_test[:32], y_test[:32],_batchSize)

In [25]:
result = parallel_model.predict_generator(small_ADG)


In [29]:
from keras.models import model_from_json
json_string = parallel_model.to_json()
with codecs.open('Audio_3LMTL.json','w', encoding = 'utf8') as outfile:
    json.dump(json_string,outfile)
with codecs.open('Audio_3LMTL_History.json','w', encoding = 'utf8') as outfile:
    json.dump(History.history,outfile)

In [17]:
Audio_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [19]:
History = Audio_model.fit_generator(
                    generator= ADG_train,
                    #steps_per_epoch=(num_training_samples // _batchsize),
                    epochs= _epchos,
                    verbose=1,
                    validation_data= ADG_test,
                    #validation_steps=(num_validation_samples // _batchsize),
                    #workers=3, use_multiprocessing=True,
                    callbacks = [AUC_test_callback,early_stopping,time_callback]
                    )

Epoch 1/100
500/500 [==============================] - 103s 207ms/step - loss: 16.0610 - Tag_1_loss: 0.7079 - Tag_2_loss: 0.5785 - Tag_3_loss: 0.5592 - Tag_4_loss: 0.5236 - Tag_5_loss: 0.2965 - Tag_6_loss: 0.3969 - Tag_7_loss: 0.4679 - Tag_8_loss: 0.4009 - Tag_9_loss: 0.2552 - Tag_10_loss: 0.3977 - Tag_11_loss: 0.3490 - Tag_12_loss: 0.1337 - Tag_13_loss: 0.4342 - Tag_14_loss: 0.3621 - Tag_15_loss: 0.5010 - Tag_16_loss: 0.2722 - Tag_17_loss: 0.3874 - Tag_18_loss: 0.3944 - Tag_19_loss: 0.3433 - Tag_20_loss: 0.2453 - Tag_21_loss: 0.3977 - Tag_22_loss: 0.3560 - Tag_23_loss: 0.1976 - Tag_24_loss: 0.3911 - Tag_25_loss: 0.3045 - Tag_26_loss: 0.2962 - Tag_27_loss: 0.2891 - Tag_28_loss: 0.3053 - Tag_29_loss: 0.3960 - Tag_30_loss: 0.0902 - Tag_31_loss: 0.3036 - Tag_32_loss: 0.2419 - Tag_33_loss: 0.3321 - Tag_34_loss: 0.2685 - Tag_35_loss: 0.3154 - Tag_36_loss: 0.2424 - Tag_37_loss: 0.3115 - Tag_38_loss: 0.1358 - Tag_39_loss: 0.2709 - Tag_40_loss: 0.2336 - Tag_41_loss: 0.2908 - Tag_42_loss: 0.226

500/500 [==============================] - 103s 206ms/step - loss: 16.1245 - Tag_1_loss: 0.7449 - Tag_2_loss: 0.5753 - Tag_3_loss: 0.5682 - Tag_4_loss: 0.5158 - Tag_5_loss: 0.2932 - Tag_6_loss: 0.4013 - Tag_7_loss: 0.4829 - Tag_8_loss: 0.4087 - Tag_9_loss: 0.2494 - Tag_10_loss: 0.3966 - Tag_11_loss: 0.3523 - Tag_12_loss: 0.1270 - Tag_13_loss: 0.4363 - Tag_14_loss: 0.3649 - Tag_15_loss: 0.5086 - Tag_16_loss: 0.2731 - Tag_17_loss: 0.3872 - Tag_18_loss: 0.3926 - Tag_19_loss: 0.3393 - Tag_20_loss: 0.2490 - Tag_21_loss: 0.3965 - Tag_22_loss: 0.3560 - Tag_23_loss: 0.1982 - Tag_24_loss: 0.3916 - Tag_25_loss: 0.3036 - Tag_26_loss: 0.2956 - Tag_27_loss: 0.2827 - Tag_28_loss: 0.3088 - Tag_29_loss: 0.4004 - Tag_30_loss: 0.0876 - Tag_31_loss: 0.3065 - Tag_32_loss: 0.2474 - Tag_33_loss: 0.3278 - Tag_34_loss: 0.2667 - Tag_35_loss: 0.3091 - Tag_36_loss: 0.2427 - Tag_37_loss: 0.3142 - Tag_38_loss: 0.1364 - Tag_39_loss: 0.2748 - Tag_40_loss: 0.2276 - Tag_41_loss: 0.2846 - Tag_42_loss: 0.2308 - Tag_43_l

KeyboardInterrupt: 

In [28]:
History.history['Timer']

[36.621094703674316,
 14.111807107925415,
 14.276816606521606,
 14.940854549407959,
 14.400823593139648,
 13.826790809631348,
 12.842734575271606,
 13.094748973846436,
 13.02274489402771,
 12.438711643218994,
 12.580719470977783,
 12.676725387573242,
 12.298703670501709,
 12.533716678619385,
 12.638723134994507,
 13.211755752563477,
 12.699726343154907,
 12.772730350494385,
 13.156752586364746,
 12.782731056213379,
 12.318704605102539,
 12.59972071647644,
 12.470713376998901]